Let's start by importing necessary Python libraries and downloading the data set.

In [ ]:
#Enables matplotlib plotting
%matplotlib inline

#Packages required for easy handling of data in python
from sklearn.model_selection import train_test_split
import pandas as pd
import wget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [ ]:
# Download the dataset. 10K events, 100 MB, no progressbar, so might need some patience.
wget.download('https://cernbox.cern.ch/index.php/s/7SPmV3ShdaQkG5C/download')

# Classifying quark and gluon jets using a deep neural network

The task of correctly identifying whether jet originated from a gluon or quark is important for almost any analysis of proton-proton collisions. This information is used to identify collision events that might contain interesting new physics and also correcting for detector effects in measured energies of jets.

As jets are objects created due to a color charge carrying particle hadronizing into a host of particles color neutral particles. This is a manifestation of the color confinement of the Strong interaction. Even though determining the particle that originated the jet is not directly visible from the end state particles, it can be inferred from the properties of the jet. Whether the initial particle was a gluon or quark affects the measurable distributions of the particles within the jet.

Let us begin by loading a sample of simulated quark and gluon jets, and inspecting its contents

In [ ]:
#Read data into a pandas DataFrame. This may also take a moment.

data = pd.read_csv("small_10K.csv")
data.head()

First we select a set of variables that are known to contain useful information, which can be used to separate quarks and gluons. All of these are mostly resulting from gluons being twice as likely to split into two compared to quarks. These variables are:

__Jet multiplicity__. The number of charged particles in the jet. One would expect there to be on average more particles in a gluon jet than a quark jet.

__Jet energy sharing variable__, $p_\text{T}D$. Describes how evenly the energy is split between the particles inside the jet. Quark jets tend to have the energy centered mostly to a few particles, whereas gluon jets have their energy more evenly spread to all the constituent particles.

__Jet minor axis__, $\sigma_{2}$. Describes the shape of the jet. The gluon jets are wider on average compared to quark jets.

Separating the relevant features from our dataframe and forming a binary target that is 1 if the jet is from a gluon and 0 if its from a quark 

In [ ]:
#Selecting the interesting features
df_=data[['QG_mult','QG_ptD','QG_axis2']].copy()

#Adding a column containing the target
df_['target']=data['isPhysG']

#we have some data quality issue here: NaNs are nasty, so clean them away
print(df_[df_['QG_ptD'].isnull()])
df_.isna().sum()
df_=df_.dropna()
df_.isna().sum()

Its always a good idea to inspect the distributions before starting to put too much effort into the machine learning task, just to see if there is something obvious visible in the data. Let us plot the variables to histograms.

In [ ]:
%matplotlib inline
columns=['QG_mult','QG_ptD','QG_axis2']
binnings=[np.arange(0.0,36.0,1.0),np.arange(0.0,1.0,0.1),np.arange(0.0,0.2,0.01)]
ind=0
fig,axes = plt.subplots(1,3,figsize=(20,10))
for column in columns:
    #print ind, binnings[ind]
    axes[ind].hist(df_[df_['target']==1][column],bins=binnings[ind],alpha=0.8,label='Gluon',density=1)
    axes[ind].hist(df_[df_['target']==0][column],bins=binnings[ind],alpha=0.8,label='Quark',density=1)
    axes[ind].set_xlabel(column)
    axes[ind].legend()
    ind=ind+1

Clearly there are some differences. For example chosing jets with more than 20 charged particles and minor axis value larger than 0.1 would contain mostly gluon jets. However we can do better by using a machine learning algorithm to learn these decision rules for us.

# Keras - Tool for Deep Learning

Keras provides a very simple interface to very powerful deep learning libraries like TensorFlow by Google. In fact, it is now integrated as a part of the TensorFlow library. It is the framework of preference for many in the particle physics community. Next we'll demonstrate that with just a few lines of code, we can produce a neural network classifier that is able to separate quark and gluon jets quite well.

In [ ]:
#Importing some functions from keras and initializing it

import tensorflow as tf
import keras.backend as K
sess = tf.Session()
K.set_session(sess)

from keras_tqdm import TQDMNotebookCallback
from keras.models import Model
from keras.layers import Input,Dense
from sklearn.utils import class_weight

#Split the data into training and test sample so that we test on data we havent
#used in training. We split target to a separate dataframe so we dont use it in
#training
train_x,test = train_test_split(df_,test_size=0.15,random_state=7)
train_y = np.array(train_x.target)
train_x = np.array(train_x.drop(['target'],axis=1))

#Defining the network shape
a_inp = Input(shape=(train_x.shape[1],))
a = Dense(50,activation='relu')(a_inp)
a = Dense(50,activation='relu')(a)
a = Dense(50,activation='relu')(a)
a_out = Dense(1,activation='sigmoid')(a)
model = Model(inputs=a_inp,outputs=a_out)
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc'])

#weight the training samples so that there is equal weight on gluon and quark jets
#even if there are different amount of them
class_weights = class_weight.compute_class_weight('balanced',np.unique(train_y),train_y[:])

#Perform the training
model.fit(train_x,train_y,epochs=10,batch_size=1024,class_weight=class_weights,
         validation_split=0.1,shuffle=True,verbose=0, callbacks=[TQDMNotebookCallback()])

Using the trained model, we can predict the classes of jets based on the three input variables. Let as create predictions for the test set and see how much our predictions separate the quark and gluon jets from each other.

In [ ]:
test.loc[:,'predictions']=model.predict(np.array(test[['QG_mult','QG_ptD','QG_axis2']].copy()))

plt.clf()
binning=np.arange(0.0,1.0,0.05)
plt.hist(test[test['target']==1]['predictions'],bins=binning,alpha=0.8,label="Gluons",density=1)
plt.hist(test[test['target']==0]['predictions'],bins=binning,alpha=0.8,label="Quarks",density=1)
plt.legend()
plt.xlabel('DNN output value')
plt.title('DNN classifier')

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve,auc
fpr,tpr, thresholds  = roc_curve(np.array(test['target']),np.array(test['predictions']))
roc_auc = auc(fpr, tpr)


plt.clf()
plt.plot(fpr,tpr,'b',label='Classifier AUC = %0.2f'% roc_auc)
plt.plot([0,1],[0,1],'k--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.legend(loc='lower right')
plt.title("Receiver Operating Characteristic")
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.savefig('roc_curve.png')

The new variable shows better separation between quark and gluon jets than any of the three variables on their own. However the strength of Deep neural networks is not in the use of this type of high level variables, but instead using very low level features and find a new representation of the data that optimizes the discrimination power. 

The dataset contains variables with prefixes Cpfcand and Npfcand. These are the particles that belong to the jet. We try adding this information and use a larger network to see if it can extract extra information from these low level features to discriminate between the quarks and gluons better.

In [ ]:
#below only works if root-file read in directly (probably because of per-particle information not written/read properly in csv-files)

from keras_preprocessing.sequence import pad_sequences
data_=data[['QG_mult','QG_ptD','QG_axis2']].copy()
particles=['PF_pT', 'PF_dR', 'PF_fromAK5Jet']
particles_=data[['PF_pT', 'PF_dR', 'PF_fromAK5Jet']].copy()
#add some of the "per-particle information" to the data frame/training
print particles_.head()

#Adding a column containing the target
data_['target']=data['isPhysG']

n_particles = 20
df2=pd.DataFrame()
#Crazy compact flattening/unrolling the particle level variables stored as vectors in the jets
for column in particles:
    column_names = [column+'_'+str(x) for x in range(n_particles)]
    df2=pd.concat([df2, pd.DataFrame(pd.DataFrame(pad_sequences(particles_[column].tolist(),
                                                                maxlen=n_particles, padding='post', dtype='float64'),
                                                  columns=column_names))], axis=1)

print data_.head()
data_=pd.concat([data_, df2], axis = 1)
print data_.head()


data_=data_.dropna()
#same cleaning as above.
# However, would want to do more: 
# - Only keep those events where PF_fromAK5Jet is true
# and [maybe; at least for charged particles] fromPV >=2 
# (charged hadron selection to suppress PU)

train_x,test = train_test_split(data_,test_size=0.15,random_state=7)
train_y = np.array(train_x.target)
train_x = np.array(train_x.drop(['target'],axis=1))

a_inp = Input(shape=(train_x.shape[1],))
a = Dense(250,activation='relu')(a_inp)
a = Dense(100,activation='relu')(a)
a = Dense(50,activation='relu')(a)
a_out = Dense(1,activation='sigmoid')(a)
model = Model(inputs=a_inp,outputs=a_out)
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc'])

model.fit(train_x,train_y,epochs=6,batch_size=1024,class_weight=class_weights,
         validation_split=0.1,shuffle=True,verbose=0, callbacks=[TQDMNotebookCallback()])

In [ ]:
test.loc[:,'predictions']=model.predict(np.array(test.drop(['target'],axis=1).copy()))

plt.clf()
binning=np.arange(0.0,1.0,0.05)
plt.hist(test[test['target']==1]['predictions'],bins=binning,alpha=0.8,label="Gluons",density=1)
plt.hist(test[test['target']==0]['predictions'],bins=binning,alpha=0.8,label="Quarks",density=1)
plt.legend()
plt.xlabel('DNN output value')
plt.title('DNN classifier')

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve,auc
fpr_,tpr_, thresholds  = roc_curve(np.array(test['target']),np.array(test['predictions']))
roc_auc_ = auc(fpr_, tpr_)


plt.clf()
plt.plot(fpr,tpr,'b',label='Before AUC = %0.2f'% roc_auc)
plt.plot(fpr_,tpr_,'r',label='After AUC = %0.2f'% roc_auc_)
plt.plot([0,1],[0,1],'k--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.legend(loc='lower right')
plt.title("Receiver Operating Characteristic")
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.savefig('roc_curve.png')


Adding new variables may increase the accuracy of the classifier. For the 10K sample here, the training sample is rather small and the input hasn't been cleaned, completely, so we even see a deteriorated performance. In genera, the performance can be further enhanced with more training statistics, more complex networks, preprocessing input variables with PCA whitening, and adding more input variables. But for this exercise this is enough.